In [14]:
from dotenv import load_dotenv
from typing_extensions import Annotated, TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field

In [3]:
load_dotenv()

True

In [4]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

In [5]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [8]:
class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

In [10]:
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [11]:
llm_with_tools = llm.bind_tools(tools)

In [12]:
query = "What is 3 * 12?"
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LfAo83W4sJZwOrlnkltcUM9H', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 87, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BF5KvD04IgU5PEimT7z7mhDpLZsaX', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-509dfa07-04ef-45b3-880d-81354d3b6389-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_LfAo83W4sJZwOrlnkltcUM9H', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87, 'output_tokens': 18, 'total_tokens': 105, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok

In [13]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_XfYnOEAZxN4fpzjildUlw6uZ',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_gBrfNs43gVBgADnwNCcJEtkX',
  'type': 'tool_call'}]

In [15]:
class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]